### 5. Make regression models

Make 3 regression models: one with "treatment period" dummy, one with "treatment city" dummy and then one with their interaction.

- Create Dummy Variables:
    - Create 2 Dummy Variables: treatment period and treatment city. 
- Remove the URLs 
- Remove € from Price column  
- Check the average prices: 
- Add a dummy variable for interaction feature of Treatment_City and Treatment_Period
- Check that descriptions are all in the same language, if not, translate descriptions from English to Spanish 
- Make OLS regressions: 
    - Regression for treatment period
    - Regression for treatment city
    - Regression for the interaction between treatment period and treatment city

Imports

In [1]:
import json
import pandas as pd
import os
from bs4 import BeautifulSoup
import time
import requests
import statsmodels.api as sm
import numpy as np 
from googletrans import Translator


ModuleNotFoundError: No module named 'pandas'

Create Dummy Variables

In [40]:
#Barcelona Dataframes
Barcelona_festival = pd.read_csv("Barcelona_Festival.csv")
Barcelona_before = pd.read_csv("Barcelona_before.csv")

# Alicante Dataframes
Alicante_festival = pd.read_csv("Alicante_festival.csv")
Alicante_before = pd.read_csv("Alicante_before.csv")



In [41]:
# Dummmy Variables Barcelona during 12-16 June 
Barcelona_festival['Treatment_Period'] = 1
Barcelona_festival['Treatment_City'] = 1 

# Dummy Variable Barcelona during 5-9  June 
Barcelona_before['Treatment_Period'] = 0
Barcelona_before['Treatment_City'] = 1 

# Dummy Variable Alicante duing 12-16 June 
Alicante_festival['Treatment_Period'] = 1
Alicante_festival['Treatment_City'] = 0

# Dummy Variable Alicante during 5-9 June 
Alicante_before['Treatment_Period'] = 0
Alicante_before['Treatment_City'] = 0


In [42]:
# combining all data in a dataframe after adding Dummy Variable
df_total = pd.concat([Barcelona_festival, Barcelona_before, Alicante_festival ,Alicante_before], ignore_index=True)
df_total.head()

,Unnamed: 0,Hotel Name,Rating,Price,Hotel URL,Descriptions,Treatment_Period,Treatment_City
0,0,Room Mate Gerard,"8,8",€ 1.100,https://www.booking.com/hotel/es/room-mate-ger...,El Room Mate Gerard en Barcelona ofrece alojam...,1,1
1,1,Melia Barcelona Sarriá,"8,2",€ 1.378,https://www.booking.com/hotel/es/melia-barcelo...,"El Meliá Barcelona Sarriá cuenta con spa, gimn...",1,1
2,2,Catalonia Diagonal Centro,"8,4",€ 799,https://www.booking.com/hotel/es/catalonia-dia...,El Catalonia Diagonal Centro se encuentra en p...,1,1
3,3,Barcelona with rooms and lofts in front of the...,"7,0",€ 224,https://www.booking.com/hotel/es/casa-nou-barc...,Barcelona with rooms and lofts in front of the...,1,1
4,4,TWO Hotel Barcelona by Axel 4* Sup- Adults Only,"8,4",€ 964,https://www.booking.com/hotel/es/two-barcelona...,El TWO Hotel Barcelona by Axel está situado en...,1,1


Clean up Dataframe

Remove the URLs

In [43]:
column_to_remove = ['Hotel URL']
df_total = df_total.drop(columns=column_to_remove)

,Unnamed: 0,Hotel Name,Rating,Price,Descriptions,Treatment_Period,Treatment_City
0,0,Room Mate Gerard,"8,8",€ 1.100,El Room Mate Gerard en Barcelona ofrece alojam...,1,1
1,1,Melia Barcelona Sarriá,"8,2",€ 1.378,"El Meliá Barcelona Sarriá cuenta con spa, gimn...",1,1
2,2,Catalonia Diagonal Centro,"8,4",€ 799,El Catalonia Diagonal Centro se encuentra en p...,1,1
3,3,Barcelona with rooms and lofts in front of the...,"7,0",€ 224,Barcelona with rooms and lofts in front of the...,1,1
4,4,TWO Hotel Barcelona by Axel 4* Sup- Adults Only,"8,4",€ 964,El TWO Hotel Barcelona by Axel está situado en...,1,1
...,...,...,...,...,...,...,...
3764,884,Beautifully presented 5 bedroom villa with WIFI,NaN,€ 1.392,Beautifully presented 5 bedroom villa with WIF...,0,0
3765,885,Villa inaya,"7,9",€ 1.398,"Villa inaya se encuentra en Alicante, a solo 7...",0,0
3766,886,Plaza Toros Apartamento,"4,0",€ 1.656,"Plaza Toros Apartamento está en Alicante, a 17...",0,0
3767,887,COZY ROOM GUAPO-ALICANTE Nearby The Beach and ...,"6,2",€ 1.325,COZY ROOM GUAPO-ALICANTE Nearby The Beach and ...,0,0


Remove € from Price column

In [44]:
df_total['Price'] = pd.to_numeric(df_total['Price'].str.replace('€', '').str.replace('.', ''), errors='coerce')
df_total

,Unnamed: 0,Hotel Name,Rating,Price,Descriptions,Treatment_Period,Treatment_City
0,0,Room Mate Gerard,"8,8",1100,El Room Mate Gerard en Barcelona ofrece alojam...,1,1
1,1,Melia Barcelona Sarriá,"8,2",1378,"El Meliá Barcelona Sarriá cuenta con spa, gimn...",1,1
2,2,Catalonia Diagonal Centro,"8,4",799,El Catalonia Diagonal Centro se encuentra en p...,1,1
3,3,Barcelona with rooms and lofts in front of the...,"7,0",224,Barcelona with rooms and lofts in front of the...,1,1
4,4,TWO Hotel Barcelona by Axel 4* Sup- Adults Only,"8,4",964,El TWO Hotel Barcelona by Axel está situado en...,1,1
...,...,...,...,...,...,...,...
3764,884,Beautifully presented 5 bedroom villa with WIFI,NaN,1392,Beautifully presented 5 bedroom villa with WIF...,0,0
3765,885,Villa inaya,"7,9",1398,"Villa inaya se encuentra en Alicante, a solo 7...",0,0
3766,886,Plaza Toros Apartamento,"4,0",1656,"Plaza Toros Apartamento está en Alicante, a 17...",0,0
3767,887,COZY ROOM GUAPO-ALICANTE Nearby The Beach and ...,"6,2",1325,COZY ROOM GUAPO-ALICANTE Nearby The Beach and ...,0,0


Check the average prices

In [45]:
# Check the mean of the prices
average_prices = df_total.groupby(['Treatment_City', 'Treatment_Period'])['Price'].mean()
print(average_prices)

Treatment_City  Treatment_Period
0               0                    634.067492
                1                    676.052511
1               0                   1081.593812
                1                   1480.578842
Name: Price, dtype: float64


Add a dummy variable for interaction feature of Treatment_City and Treatment_Period

In [46]:
df_total['Interaction_Period_City'] = df_total['Treatment_Period']  * df_total['Treatment_City']
df_total

,Unnamed: 0,Hotel Name,Rating,Price,Descriptions,Treatment_Period,Treatment_City,Interaction_Period_City
0,0,Room Mate Gerard,"8,8",1100,El Room Mate Gerard en Barcelona ofrece alojam...,1,1,1
1,1,Melia Barcelona Sarriá,"8,2",1378,"El Meliá Barcelona Sarriá cuenta con spa, gimn...",1,1,1
2,2,Catalonia Diagonal Centro,"8,4",799,El Catalonia Diagonal Centro se encuentra en p...,1,1,1
3,3,Barcelona with rooms and lofts in front of the...,"7,0",224,Barcelona with rooms and lofts in front of the...,1,1,1
4,4,TWO Hotel Barcelona by Axel 4* Sup- Adults Only,"8,4",964,El TWO Hotel Barcelona by Axel está situado en...,1,1,1
...,...,...,...,...,...,...,...,...
3764,884,Beautifully presented 5 bedroom villa with WIFI,NaN,1392,Beautifully presented 5 bedroom villa with WIF...,0,0,0
3765,885,Villa inaya,"7,9",1398,"Villa inaya se encuentra en Alicante, a solo 7...",0,0,0
3766,886,Plaza Toros Apartamento,"4,0",1656,"Plaza Toros Apartamento está en Alicante, a 17...",0,0,0
3767,887,COZY ROOM GUAPO-ALICANTE Nearby The Beach and ...,"6,2",1325,COZY ROOM GUAPO-ALICANTE Nearby The Beach and ...,0,0,0


Check that descriptions are all in the same language

In [47]:
# Might need to pip install some libraries
#!pip install langdetect 
# !pip install googletrans==4.0.0-rc1

# Detect which language the description is in 
from langdetect import detect
df_total['language'] = df_total['Descriptions'].apply(lambda x: detect(x) if type(x) == str else None)
df_total


,Unnamed: 0,Hotel Name,Rating,Price,Descriptions,Treatment_Period,Treatment_City,Interaction_Period_City,language
0,0,Room Mate Gerard,"8,8",1100,El Room Mate Gerard en Barcelona ofrece alojam...,1,1,1,es
1,1,Melia Barcelona Sarriá,"8,2",1378,"El Meliá Barcelona Sarriá cuenta con spa, gimn...",1,1,1,es
2,2,Catalonia Diagonal Centro,"8,4",799,El Catalonia Diagonal Centro se encuentra en p...,1,1,1,es
3,3,Barcelona with rooms and lofts in front of the...,"7,0",224,Barcelona with rooms and lofts in front of the...,1,1,1,es
4,4,TWO Hotel Barcelona by Axel 4* Sup- Adults Only,"8,4",964,El TWO Hotel Barcelona by Axel está situado en...,1,1,1,es
...,...,...,...,...,...,...,...,...,...
3764,884,Beautifully presented 5 bedroom villa with WIFI,NaN,1392,Beautifully presented 5 bedroom villa with WIF...,0,0,0,es
3765,885,Villa inaya,"7,9",1398,"Villa inaya se encuentra en Alicante, a solo 7...",0,0,0,es
3766,886,Plaza Toros Apartamento,"4,0",1656,"Plaza Toros Apartamento está en Alicante, a 17...",0,0,0,es
3767,887,COZY ROOM GUAPO-ALICANTE Nearby The Beach and ...,"6,2",1325,COZY ROOM GUAPO-ALICANTE Nearby The Beach and ...,0,0,0,es


In [34]:
# Count total descriptions in Spanish and in English 
english_count = df_total[df_total['language'] == 'en'].shape[0]
spanish_count = df_total[df_total['language'] == 'es'].shape[0]
print(f'Number of descriptions in English: {english_count}')
print(f'Number of descriptions in Spanish: {spanish_count}')


Number of descriptions in English: 17
Number of descriptions in Spanish: 3751


In [39]:
# Display the rows where the language is English
english_rows = df_total[df_total['language'] == 'en']
print(english_rows)

      Unnamed: 0                         Hotel Name Rating  Price  \
150          150        Aparthotel Arai 4* Superior    9,0   1455   
160          160   Torre Melina, a Gran Meliá Hotel    NaN   1995   
288          288       Àmfores Boutique Guest House    8,1    926   
410          410             Sonder La Casa del Sol    7,9   1126   
461          461              TOC Hotel Las Ramblas    8,3   1354   
469          469   DO Plaça Reial powered by Sonder    8,4   1980   
481          481               Ant Hostel Barcelona    7,0    993   
544          544          Twentytu Hostel Barcelona    7,7   1993   
1222         220       Àmfores Boutique Guest House    8,1    766   
1289         287        Aparthotel Arai 4* Superior    9,0   1489   
1305         303   Torre Melina, a Gran Meliá Hotel    NaN   2215   
1334         332          Twentytu Hostel Barcelona    7,7   2353   
1575         573             Sonder La Casa del Sol    7,9   1145   
1621         619   DO Plaça Reial 

Translate descriptions from English to Spanish 

In [36]:
def translate_to_spanish(text):
    """
    Translates the descriptions from English to Spanish. 

    Parameters: 
        - text (str): The input description to be translated. 

    Returns: 
        - str: The translated description in Spanish. 
    """
    translator = Translator()
    translation = translator.translate(text, src='en', dest='es')
    return translation.text

english_descriptions = df_total[df_total['language'] == 'en']['Descriptions']

# Translate English descriptions to Spanish
df_total.loc[df_total['language'] == 'en', 'Descriptions'] = english_descriptions.apply(translate_to_spanish)

print(df_total)


      Unnamed: 0                                         Hotel Name Rating  \
0              0                                   Room Mate Gerard    8,8   
1              1                             Melia Barcelona Sarriá    8,2   
2              2                          Catalonia Diagonal Centro    8,4   
3              3  Barcelona with rooms and lofts in front of the...    7,0   
4              4    TWO Hotel Barcelona by Axel 4* Sup- Adults Only    8,4   
...          ...                                                ...    ...   
3764         884    Beautifully presented 5 bedroom villa with WIFI    NaN   
3765         885                                        Villa inaya    7,9   
3766         886                            Plaza Toros Apartamento    4,0   
3767         887  COZY ROOM GUAPO-ALICANTE Nearby The Beach and ...    6,2   
3768         888                                 Victoria Hospedaje    9,0   

      Price                                       Descriptions 

In [38]:
# Check if it worked (should all come out in Spanish now )
english_rows = df_total[df_total['language'] == 'en']

# Display the rows where the language is English
print(english_rows)


      Unnamed: 0                         Hotel Name Rating  Price  \
150          150        Aparthotel Arai 4* Superior    9,0   1455   
160          160   Torre Melina, a Gran Meliá Hotel    NaN   1995   
288          288       Àmfores Boutique Guest House    8,1    926   
410          410             Sonder La Casa del Sol    7,9   1126   
461          461              TOC Hotel Las Ramblas    8,3   1354   
469          469   DO Plaça Reial powered by Sonder    8,4   1980   
481          481               Ant Hostel Barcelona    7,0    993   
544          544          Twentytu Hostel Barcelona    7,7   1993   
1222         220       Àmfores Boutique Guest House    8,1    766   
1289         287        Aparthotel Arai 4* Superior    9,0   1489   
1305         303   Torre Melina, a Gran Meliá Hotel    NaN   2215   
1334         332          Twentytu Hostel Barcelona    7,7   2353   
1575         573             Sonder La Casa del Sol    7,9   1145   
1621         619   DO Plaça Reial 

OLS REGRESSIONS

In [10]:
# Regression for Treatment Period 
const_period= sm.add_constant(df_total['Treatment_Period'])
model_per = sm.OLS(df_total['Price'], const_period)
ols_period = model_per.fit()
print(ols_period.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     85.25
Date:                Fri, 02 Feb 2024   Prob (F-statistic):           4.28e-20
Time:                        13:56:52   Log-Likelihood:                -30438.
No. Observations:                3769   AIC:                         6.088e+04
Df Residuals:                    3767   BIC:                         6.089e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              871.2020     17.898  

In [11]:
# Regression for Treatment City 
const_city= sm.add_constant(df_total['Treatment_City'])
model_city = sm.OLS(df_total['Price'], const_city)
ols_city = model_city.fit()
print(ols_city.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.158
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     705.3
Date:                Fri, 02 Feb 2024   Prob (F-statistic):          1.37e-142
Time:                        13:56:57   Log-Likelihood:                -30156.
No. Observations:                3769   AIC:                         6.032e+04
Df Residuals:                    3767   BIC:                         6.033e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            654.9054     17.193     38.

In [14]:
# Regression for the interaction between treatment period and treatment city 
model_interaction = sm.OLS(df_total['Price'], sm.add_constant(df_total[['Treatment_Period', 'Treatment_City', 'Interaction_Period_City']]))
ols_interaction = model_interaction.fit()
print(ols_interaction.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.192
Model:                            OLS   Adj. R-squared:                  0.192
Method:                 Least Squares   F-statistic:                     298.6
Date:                Fri, 02 Feb 2024   Prob (F-statistic):          6.67e-174
Time:                        13:57:15   Log-Likelihood:                -30078.
No. Observations:                3769   AIC:                         6.016e+04
Df Residuals:                    3765   BIC:                         6.019e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     